# FLAX Convolution Neural Network Example - Interactive API

Run this jupyter notebook on a virtual environment.

In [2]:
import os
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.8'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
# !pip install --upgrade -q "jax[cuda11_cudnn805]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# !pip install --upgrade -q git+https://github.com/google/flax.git

In [ ]:
# !pip install flax==0.4.1 ml_collections optax jax==0.3.13 -q
# # jaxlib==0.3.10 -q

GPU version of JAX. Pick the jax version compatible with the CUDA and cuDNN pre-installed.

In [ ]:
# !pip install --upgrade pip # Careful with the pip upgrade, it may cause a package dependency related problems during OpenFL workflow execution.

# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# Note: wheels only available on linux.
# !pip install -U jaxlib==0.3.10+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# Installs the wheel compatible with Cuda >= 11.4 and cudnn >= 8.2
# !pip install -U jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

# Installs the wheel compatible with Cuda >= 11.1 and cudnn >= 8.0.5
# !pip install "jax[cuda11_cudnn805]" -f https://storage.googleapis.com/jax-releases/jax_releases.html

In [ ]:
# Without either of the below flags, JAX XLA raised CUDA_OUT_OF_MEMORY exception.
# JAX XLA pre-allocates 90% of the GPU at start

# Below flag to restrict max GPU allocation to 80%
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=.8

# OR

# set XLA_PYTHON_CLIENT_PREALLOCATE to false to incrementally allocate GPU memory as and when required. But can take entire GPU by the end.
# %env XLA_PYTHON_CLIENT_PREALLOCATE=false


In [ ]:
# %pip install tensorflow==2.8.1


In [ ]:
# %pip install tensorflow_datasets ml_collections


In [3]:
%env

{'SHELL': '/bin/bash',
 'COLORTERM': 'truecolor',
 'no_proxy': 'localhost,127.0.0.0/8,10.0.0.0/8,.intel.com',
 'TERM_PROGRAM_VERSION': '1.69.2',
 'LANGUAGE': 'en_IN:en',
 'PWD': '/home/sunilach/openfl/forked-intel-openfl/openfl-tutorials/interactive_api/Flax_CNN_CIFAR',
 'LOGNAME': 'sunilach',
 'XDG_SESSION_TYPE': 'tty',
 'ftp_proxy': 'ftp://proxy.iind.intel.com:1080/',
 'MOTD_SHOWN': 'pam',
 'HOME': '/home/sunilach',
 'LANG': 'en_US.UTF-8',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.zst=01;31:*.tzst=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31

In [4]:
from flax import linen as nn
from flax.metrics import tensorboard
from flax.training import train_state
import jax
import jax.numpy as jnp
import ml_collections
import optax
import tensorflow_datasets as tfds
import logging

/home/sunilach/openflenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# # Both the MSE function are optimal and accurate in terms of correctness.

# # Calculate MSE approach 1
# def mse_loss_function1(W, X, y):
#     y_pred = jnp.dot(X, W)
#     mse_error = y_pred - y
#     return jnp.mean(jnp.square(mse_error))

# # Calculate MSE approach 2
# def mse_loss_function2(W, X, Y):
#     def squared_error(x, y):
#         y_pred = jnp.dot(x, W)
#         return jnp.inner(y-y_pred, y-y_pred)|
#     vectorized_square_error = jax.vmap(squared_error)
#     return jnp.mean(vectorized_square_error(X, Y), axis=0)

# # Weight update, JAX compiled function. Consequent executions are way faster!!!.
# def update(W, x, y, lr):
#     W = W - lr * jax.grad(mse_loss_function1)(W, x, y)
#     return W

In [5]:
editor_relpaths = ('configs/default.py', 'train.py')

In [6]:
@jax.jit
def apply_model(state, images, labels):
    """Computes gradients, loss and accuracy for a single batch."""

    def loss_fn(params):
        logits = state.apply_fn({'params': params}, images)
        one_hot = jax.nn.one_hot(labels, 10)
        loss = jnp.mean(optax.softmax_cross_entropy(logits=logits, labels=one_hot))
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (loss, logits), grads = grad_fn(state.params)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    return grads, loss, accuracy


@jax.jit
def update_model(state, grads):
    return state.apply_gradients(grads=grads)

In [307]:
def train_epoch(state, train_ds, batch_size, rng):
    """Train for a single epoch."""
    train_ds_size = len(train_ds['image'])
    steps_per_epoch = train_ds_size // batch_size

    perms = jax.random.permutation(rng, len(train_ds['image']))
    perms = perms[:steps_per_epoch * batch_size]  # skip incomplete batch
    perms = perms.reshape((steps_per_epoch, batch_size))

    epoch_loss = []
    epoch_accuracy = []

    for perm in perms:
        batch_images = train_ds['image'][perm, ...]
        batch_labels = train_ds['label'][perm, ...]
        grads, loss, accuracy = apply_model(state, batch_images, batch_labels)
        print(grads)
        break
        state = update_model(state, grads)
        epoch_loss.append(loss)
        epoch_accuracy.append(accuracy)
        
    train_loss = np.mean(epoch_loss)
    train_accuracy = np.mean(epoch_accuracy)
    return state, train_loss, train_accuracy

In [308]:
def train_and_evaluate(config: ml_collections.ConfigDict,
                       workdir: str) -> train_state.TrainState:
    """Execute model training and evaluation loop.
    Args:
        config: Hyperparameter configuration for training and evaluation.
        workdir: Directory where the tensorboard summaries are written to.
    Returns:
        The train state (which includes the `.params`).
    """
    
    train_ds, test_ds = get_datasets()
    rng = jax.random.PRNGKey(0)

    summary_writer = tensorboard.SummaryWriter(workdir)
    summary_writer.hparams(dict(config))

    rng, init_rng = jax.random.split(rng)
    state = create_train_state(init_rng, config)

    for epoch in range(1, config.num_epochs + 1):
        rng, input_rng = jax.random.split(rng)
        state, train_loss, train_accuracy = train_epoch(state, train_ds,
                                                        config.batch_size,
                                                        input_rng)
        _, test_loss, test_accuracy = apply_model(state, test_ds['image'],
                                                  test_ds['label'])

        logging.info(
            'epoch:% 3d, train_loss: %.4f, train_accuracy: %.2f, test_loss: %.4f, test_accuracy: %.2f'
            % (epoch, train_loss, train_accuracy * 100, test_loss,
               test_accuracy * 100))

        summary_writer.scalar('train_loss', train_loss, epoch)
        summary_writer.scalar('train_accuracy', train_accuracy, epoch)
        summary_writer.scalar('test_loss', test_loss, epoch)
        summary_writer.scalar('test_accuracy', test_accuracy, epoch)

    summary_writer.flush()
    return state

In [9]:
# #Using jit decorator for GPU acceleration for entire function
# @jax.jit
# def train_step(optimizer, batch):
#     def loss_fn(model):
#         preds = model(batch['image'])
#         loss = cross_entropy_loss(preds, batch['label'])
#         return loss, preds
#     grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
#     (_, preds), grad = grad_fn(optimizer.target)
#     optimizer = optimizer.apply_gradient(grad)
#     return optimizer

# @jax.jit
# def eval_step(model, batch):
#     preds = model(batch['image'])
#     return compute_metrics(preds, batch['label'])

# def eval_model(model, test_ds):
#     metrics = eval_step(model, test_ds)
#     metrics = jax.device_get(metrics)
#     summary = jax.tree_map(lambda x: x.item(), metrics)
#     return summary['loss'], summary['accuracy']

In [309]:
def get_datasets():
    """Load MNIST train and test datasets into memory."""
    ds_builder = tfds.builder('mnist')
    ds_builder.download_and_prepare()
    train_ds = tfds.as_numpy(ds_builder.as_dataset(split='train', batch_size=-1))
    test_ds = tfds.as_numpy(ds_builder.as_dataset(split='test', batch_size=-1))
    train_ds['image'] = jnp.float32(train_ds['image']) / 255.
    test_ds['image'] = jnp.float32(test_ds['image']) / 255.
    return train_ds, test_ds

In [338]:
# rng = jax.random.PRNGKey(0)
# rng, init_rng = jax.random.split(rng)
# cnn = CNN()
# params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params'].unfreeze()

In [341]:
def create_train_state(rng, config):
    """Creates initial `TrainState`."""
    cnn = CNN()
    params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params'].unfreeze()
    tx = optax.sgd(config.learning_rate, config.momentum)
    return train_state.TrainState.create(
      apply_fn=cnn.apply, params=params, tx=tx)

In [342]:
class CNN(nn.Module):
    """A simple CNN model."""
    
    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = nn.Conv(features=64, kernel_size=(3, 3))(x)
        x = nn.relu(x)
        x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        x = x.reshape((x.shape[0], -1))  # flatten
        x = nn.Dense(features=256)(x)
        x = nn.relu(x)
        x = nn.Dense(features=10)(x)
        return x

In [13]:
train_ds, test_ds = get_datasets()

Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


2022-08-09 15:09:37.394065: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [14]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# # Helper functions for images.

# def show_img(img, ax=None, title=None):
#     """Shows a single image."""
#     if ax is None:
#         ax = plt.gca()
#         ax.imshow(img[..., 0], cmap='gray')
#         ax.set_xticks([])
#         ax.set_yticks([])
#     if title:
#         ax.set_title(title)

# def show_img_grid(imgs, titles):
#     """Shows a grid of images."""
#     n = int(np.ceil(len(imgs)**.5))
#     _, axs = plt.subplots(n, n, figsize=(3 * n, 3 * n))
#     for i, (img, title) in enumerate(zip(imgs, titles)):
#         show_img(img, axs[i // n][i % n], title)

In [ ]:
# show_img_grid(
#     [train_ds['image'][idx] for idx in range(25)],
#     [f'label={train_ds["label"][idx]}' for idx in range(25)],
# )

In [343]:
from configs import default as config_lib
config = config_lib.get_config()

In [ ]:
# cnn = CNN()
# params = cnn.init(rng, jnp.ones([1, 28, 28, 1]))['params']

In [344]:
config.num_epochs = 3
models = {}
for momentum in (0.8, 0.9, 0.95):
    name = f'momentum={momentum}'
    config.momentum = momentum
    state = train_and_evaluate(config, workdir=f'./models/{name}')
    break
    models[name] = state.params

{'Conv_0': {'bias': DeviceArray([-2.5294956e-03,  1.0617384e-03,  1.0871092e-03,
             -7.5664517e-04, -1.8678233e-04, -4.9029361e-04,
             -4.4167665e-04, -7.9249972e-03, -9.1656536e-04,
             -1.8964002e-03, -2.1935222e-03,  7.8311907e-03,
              2.1288046e-03, -1.6735045e-03, -3.3306577e-03,
             -1.0311848e-03,  7.7862656e-03,  2.1576872e-03,
             -7.2822906e-03, -1.5483726e-03, -6.3303817e-04,
             -4.6299753e-04,  4.3941289e-03, -5.3220720e-04,
             -8.3184766e-04,  7.8265844e-03, -3.4356546e-03,
             -3.6323036e-05, -2.5688871e-03,  1.1922442e-03,
             -5.2049058e-04, -4.6177106e-03], dtype=float32), 'kernel': DeviceArray([[[[-6.59753452e-04,  9.93206748e-04,  4.25191654e-04,
                -1.60608336e-03, -1.48017437e-03,  1.05801504e-03,
                -2.38798209e-04, -2.88684689e-03, -9.36191296e-04,
                -1.06081145e-03, -3.68145120e-04,  4.18583397e-03,
                -1.70943735e-0

/home/sunilach/openflenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/sunilach/openflenv/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'Conv_0': {'bias': DeviceArray([-0.00429211,  0.00014817,  0.00244336,  0.00265356,
             -0.00806753,  0.00110937, -0.00016437, -0.00045536,
             -0.00082569,  0.00109708,  0.00296827,  0.00832468,
              0.00184548, -0.00093308, -0.00029506, -0.00064374,
              0.0170198 ,  0.00206642, -0.00162658, -0.00632473,
             -0.00038288, -0.00032191,  0.00537447,  0.00046672,
             -0.00196092,  0.00701693, -0.00064962, -0.00046556,
             -0.00377867,  0.00299433, -0.00460592, -0.00390522],            dtype=float32), 'kernel': DeviceArray([[[[-1.74214155e-03,  4.18937649e-04,  2.15712716e-04,
                -9.75224306e-04, -2.56598252e-03,  1.26618624e-03,
                -2.05858014e-04,  5.63990034e-04, -2.27882992e-08,
                 8.98707483e-04,  3.82827688e-03,  4.17585997e-03,
                -9.91377165e-05,  2.81037384e-04,  4.01121855e-04,
                -4.87552752e-04,  7.82713573e-03,  2.50424747e-03,
                -1.2

In [326]:
new_model.apply_gradients(grads=uf_params)

ValueError: Expected dict, got FrozenDict({
    Conv_0: {
        bias: DeviceArray([ 1.0808865e-31, -4.5855012e-02,  2.8046457e-02,
                      5.4478245e-32,  1.4003498e-31,  9.9151448e-06,
                      7.7850878e-02,  2.9201466e-31,  1.7564533e-31,
                      3.2798093e-02,  1.6265796e-31,  1.7444642e-31,
                      1.7386822e-02,  4.9014372e-31,  9.8282407e-32,
                      2.8425191e-02,  2.1398419e-31,  7.5427423e-32,
                      7.2153812e-32,  3.7461914e-02, -1.4826582e-02,
                      7.1077035e-03, -3.4116551e-02,  1.0147342e-31,
                      4.1000829e-31,  9.8198112e-32, -1.5755799e-03,
                     -3.2646570e-02, -3.2302715e-02, -5.4963474e-04,
                      5.0304699e-32, -3.0913241e-03], dtype=float32),
        kernel: DeviceArray([[[[ 1.88760765e-32, -1.06703984e-02,  1.37033593e-02,
                         9.39276337e-33,  4.76551170e-32, -3.07930395e-06,
                  

In [346]:
param =jax.tree_util.tree_map(np.array, state.params)

In [44]:
!pip install flatdict

In [45]:
import flatdict

In [66]:
d = flatdict.FlatDict(param, delimiter='.')

In [96]:
'.'.join(filter(None, ['', 'Conv_0', 'kernel', '', 'Lol']))

'Conv_0.kernel.Lol'

In [350]:
weight_dict = dict()

In [351]:
alias = 'param'
delim = '.'

In [352]:
param = state.params

In [353]:
for layer_name, param_obj in param.items():
    for param_name, value in param_obj.items():
        key = delim.join([alias, layer_name, param_name])
        weight_dict[key] = value

In [354]:
for k, v in weight_dict.items():
    print(type(k), type(v))
    break

<class 'str'> <class 'jaxlib.xla_extension.DeviceArray'>


In [355]:
model = state

In [356]:
def _get_weights_dict(obj, prefix='', suffix=''):
    """
    Get the dictionary of weights.

    Parameters
    ----------
    obj : Model or Optimizer
        The target object that we want to get the weights.

    Returns
    -------
    dict
        The weight dictionary.
    """
    weights_dict = dict()
    delim = '.'
    for layer_name, param_obj in obj.items():
        for param_name, value in param_obj.items():
            key = delim.join(filter(None, [prefix, layer_name, param_name, suffix]))
            weights_dict[key] = value

    return weights_dict

In [362]:
model_params = jax.tree_util.tree_map(np.array, model.params)
model_opt_state = jax.tree_util.tree_map(np.array, model.opt_state)[0][0]

In [364]:
opt_dict = _get_weights_dict(model_opt_state, 'opt', suffix)

In [366]:
suffix = ''

In [367]:
model_params = jax.tree_util.tree_map(np.array, model.params)
params_dict = _get_weights_dict(model_params, 'param', suffix)

if model.opt_state is not None:
    model_opt_state = jax.tree_util.tree_map(np.array, model.opt_state)[0][0]
    opt_dict = _get_weights_dict(model_opt_state, 'opt', suffix)
    params_dict.update(opt_dict)

In [368]:
for k, v in params_dict.items():
    print(k)

param.Conv_0.bias
param.Conv_0.kernel
param.Conv_1.bias
param.Conv_1.kernel
param.Dense_0.bias
param.Dense_0.kernel
param.Dense_1.bias
param.Dense_1.kernel
opt.Conv_0.bias
opt.Conv_0.kernel
opt.Conv_1.bias
opt.Conv_1.kernel
opt.Dense_0.bias
opt.Dense_0.kernel
opt.Dense_1.bias
opt.Dense_1.kernel


In [369]:
new_model = state

In [370]:
from flax.core.frozen_dict import freeze, unfreeze

In [250]:
{'bias': new_model.params['Conv_0']['bias'].at[0].set(1.10101)}

{'bias': DeviceArray([ 1.10101   , -0.09015119, -0.13249867, -1.272739  ,
              -2.9447565 , -0.42616746,  0.52555424, -6.1838517 ,
              -1.3973562 , -0.52935064, -3.4368992 , -3.6981347 ,
              -0.25468838, -4.1671534 , -2.1030016 , -0.14081043,
              -4.5040197 , -1.5975306 , -1.5151801 , -2.4758625 ,
              -0.09489761,  0.02707327, -0.52436036, -2.1598485 ,
              -3.6401956 , -2.0718012 , -1.2368054 , -0.13675065,
              -0.36861733, -0.40148488, -1.0966164 , -0.83448386],            dtype=float32)}

In [271]:
grads

NameError: name 'grads' is not defined

In [259]:
params_ = jax.tree_util.tree_map(np.array, model.params)

In [266]:
params_ = unfreeze(params_)

In [267]:
params_['Conv_0']['bias']= 1.10101

In [270]:
model.params = freeze(params_)

FrozenInstanceError: cannot assign to field 'params'

In [235]:
new_model.params['Conv_0']['bias'] = unfreeze(new_model.params['Conv_0']['bias'].at[0].set(1.10101)

ValueError: FrozenDict is immutable.

In [375]:
new_model

TrainState(step=0, apply_fn=<bound method Module.apply of CNN()>, params={'Conv_0': {'bias': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
             0., 0.], dtype=float32), 'kernel': DeviceArray([[[[ 3.32822055e-01, -2.82645106e-01, -1.80686682e-01,
                -6.79024830e-02, -7.08380640e-01,  2.77156025e-01,
                 2.83575952e-01,  5.28992653e-01, -3.09524149e-01,
                 5.76644480e-01,  3.28146875e-01, -7.18251020e-02,
                -4.51019973e-01, -5.43700039e-01, -3.07050906e-02,
                -4.92292970e-01,  7.26195991e-01,  1.11178890e-01,
                 3.07978690e-01, -7.01449215e-01, -7.72421658e-02,
                -4.72714245e-01,  1.41713738e-01, -1.60780087e-01,
                -1.76688954e-01,  1.37713805e-01, -5.79014122e-02,
                 1.17106706e-01,  3.02598029e-01,  2.02198133e-01,
                 3.30374956e-01,  7.1303147

In [380]:
params_dict = jax.tree_util.tree_map(jnp.array, params_dict)

In [381]:
params_dict

{'opt.Conv_0.bias': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0.], dtype=float32),
 'opt.Conv_0.kernel': DeviceArray([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0.]],
 
               [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0.]],
 
               [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0.]]],
 
 
              [[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                 0., 0., 0., 0.]],
 
               [[0., 0., 0., 0., 0., 0

In [389]:
ja = jnp.array([0.80, 0.80, 0.88, 0.887, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
              0., 0.])

In [390]:
params_dict['opt.Conv_0.bias'] = ja

In [397]:
prefix = 'param'
delim = '.'
for layer_name, param_obj in model_params.items():
    for param_name, value in param_obj.items():
        key = delim.join(filter(None, [prefix, layer_name, param_name]))
        if key in params_dict:
            model_params[layer_name][param_name] = params_dict[key]

if model.opt_state is not None:
    prefix = 'opt'
    for layer_name, param_obj in model_opt_state.items():
        for param_name, value in param_obj.items():
            key = delim.join(filter(None, [prefix, layer_name, param_name]))
            if key in params_dict:
                model_opt_state[layer_name][param_name] = params_dict[key]

# new_model.params = jax.tree_util.tree_map(jnp.array, model_params)
new_model.opt_state[0][0] = freeze(jax.tree_util.tree_map(jnp.array, model_opt_state))

TypeError: 'TraceState' object does not support item assignment

In [ ]:
optstate1 = jax.tree_util.tree_map(np.array, state.opt_state)

In [ ]:
jax.tree_util.tree_map(jnp.array, optstate1)

In [ ]:
for i in state.params:
    print(i)

In [ ]:
model_weight_names = [weight for weight in state.params]

In [ ]:
import copy

In [ ]:
tensor_dict = copy.deepcopy(state.params)

In [ ]:
model_weight_names

In [ ]:
model_weights_dict = { name: tensor_dict[name] for name in model_weight_names }

In [ ]:
for name in model_weight_names:
    print(name)

In [ ]:
state1 = jax.tree_map(np.array, state)

In [ ]:
# Find all mistakes in testset.
logits = CNN().apply({'params': state.params}, test_ds['image'])


In [ ]:
error_idxs, = jnp.where(test_ds['label'] != logits.argmax(axis=1))
len(error_idxs) / len(logits)

In [ ]:
show_img_grid(
    [test_ds['image'][idx] for idx in error_idxs[:25]],
    [f'pred={logits[idx].argmax()}' for idx in error_idxs[:25]],
)

In [ ]:
# class LinearRegression:
#     def __init__(self, n_feat: int) -> None:
#         self.weights = jnp.ones(n_feat)
    
#     def mse(self, X, y) -> float:
#         return mse_loss_function1(self.weights, X, y)
 
#     def predict(self, X):
#         return jnp.dot(X, self.weights)
    
#     def fit(self, X, Y, n_epochs : int, learning_rate : int, silent : bool) -> None:
        
#         # Speed up weight updates with consecutive calls to jitted `update` function.
#         update_weights = jax.jit(update)
        
#         start_time = time.time()
#         print('Training Loss at start: ', self.mse(X, Y))
#         for i in range(n_epochs):
#             self.weights = update_weights(self.weights, X, Y, learning_rate)
#             if i % int(n_epochs/10) == 0 and not silent:
#                 print(str(i), 'Training Loss: ', self.mse(X, Y))

#         print("--- %s seconds ---" % (time.time() - start_time))

    

In [ ]:
_, init_params = CNN().init_by_shape(random.PRNGKey(0), [((1, 32, 32, 3), jnp.float32)])
model = nn.Model(CNN, init_params)
optimizer = optim.Adam(learning_rate=learning_rate, beta1=beta, beta2 = beta_2).create(model)

#Jit Pre-compilation

In [ ]:
train_step(optimizer, train_ds)

In [ ]:
eval_model(optimizer.target, test_ds)

In [ ]:
def train(train_ds, test_ds, model, optimizer):

    batch_size = 128
    num_epochs = 10
    learning_rate = 0.001
    beta = 0.9
    beta_2 = 0.999
    loss = 0
    accuracy = 0
    
    start_time = time.monotonic()
    
    for epoch in range(1, num_epochs + 1):
        train_time = 0
        start_time_3 = time.monotonic()
        batch_gen = tfds.as_numpy(train_ds)
        for batch in batch_gen:
            start_time_step = time.monotonic()
            optimizer = train_step(optimizer, batch)
            train_time += time.monotonic() - start_time_step
            
        flax_step = time.monotonic() - start_time_3
        
        start_time_2 = time.monotonic()
        loss, accuracy = eval_model(optimizer.target, test_ds)
        flax_inf = time.monotonic() - start_time_2
        
        print('eval epoch: %d, epoch: %.2fs, actual_training: %.2fs, validation: %.2fs, loss: %.4f, accuracy: %.2f' % 
              (epoch, flax_step, train_time, flax_inf, loss, accuracy * 100))
        
    flax_time = time.monotonic() - start_time
    return optimizer, flax_time, accuracy, flax_inf

In [ ]:
_, flax_time, flax_acc, flax_inf = train(train_ds, test_ds, model, optimizer)

# JAX Linear Regression with federation

## Connect to a Federation

In [ ]:
# Create a federation
from openfl.interface.interactive_api.federation import Federation

# please use the same identificator that was used in signed certificate
client_id = 'frontend'
director_node_fqdn = 'localhost'
director_port = 50050

federation = Federation(
    client_id=client_id,
    director_node_fqdn=director_node_fqdn,
    director_port=director_port,
    tls=False
)

In [ ]:
shard_registry = federation.get_shard_registry()
shard_registry

### Initialize Data Interface

In [ ]:
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface, FLExperiment

class LinearRegressionDataSet(DataInterface):
    def __init__(self, **kwargs):
        """Initialize DataLoader."""
        self.kwargs = kwargs

    @property
    def shard_descriptor(self):
        """Return shard descriptor."""
        return self._shard_descriptor
    
    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.
        
        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor
        self.train_set = shard_descriptor.get_dataset('train')
        self.val_set = shard_descriptor.get_dataset('val')
        
    def get_train_loader(self, **kwargs):
        """Output of this method will be provided to tasks with optimizer in contract."""
        return self.train_set

    def get_valid_loader(self, **kwargs):
        """Output of this method will be provided to tasks without optimizer in contract."""
        return self.val_set

    def get_train_data_size(self):
        """Information for aggregation."""
        return len(self.train_set)

    def get_valid_data_size(self):
        """Information for aggregation."""
        return len(self.val_set)
    
lin_reg_dataset = LinearRegressionDataSet()

### Define Model Interface

In [ ]:
framework_adapter = 'custom_adapter.CustomFrameworkAdapter'

# LinearRegression class accepts a parameter n_features. Should be same as `sample_shape` from `director_config.yaml`
fed_model = LinearRegression(1)
MI = ModelInterface(model=fed_model, optimizer=None, framework_plugin=framework_adapter)

# Save the initial model state
initial_model = LinearRegression(1)

### Register Tasks
We need to employ a trick reporting metrics. OpenFL decides which model is the best based on an *increasing* metric.

In [ ]:
TI = TaskInterface()

@TI.add_kwargs(**{'lr': 0.01,
                   'epochs': 101})
@TI.register_fl_task(model='my_model', data_loader='train_data', \
                     device='device', optimizer='optimizer')     
def train(my_model, train_data, optimizer, device, lr, epochs):
    X, Y = train_data[:,:-1], train_data[:,-1]
    my_model.fit(X, Y, epochs, lr, silent=False)
    return {'train_MSE': my_model.mse(X, Y),}

@TI.register_fl_task(model='my_model', data_loader='val_data', device='device')
def validate(my_model, val_data, device):
    X, Y = val_data[:,:-1], val_data[:,-1] 
    return {'validation_MSE': my_model.mse(X, Y),}

### Run the federation

In [ ]:
experiment_name = 'jax_linear_regression_experiment'
fl_experiment = FLExperiment(federation=federation, experiment_name=experiment_name)

In [ ]:
fl_experiment.start(model_provider=MI,
                    task_keeper=TI,
                    data_loader=lin_reg_dataset,
                    rounds_to_train=2)

In [ ]:
fl_experiment.stream_metrics()

# JAX Linear Regression without federation (Optional Simulation)

In [ ]:
!pip install matplotlib scikit-learn -q

In [ ]:
# Imports for running JAX Linear Regression example without OpenFL.

import matplotlib.pyplot as plt

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 7, 5

from jax import make_jaxpr
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

#### Simple Linear Regression
<img src="https://www.analyticsvidhya.com/wp-content/uploads/2016/01/eq5-1.png" width="500">



In [ ]:
# create a dataset with n_features
X, y = make_regression(n_samples=100, n_features=1, noise=10, random_state=42)

# Train test split - Default 0.75/0.25
X, X_test, y, y_test = train_test_split(X, y, random_state=42)

Visualize data distribution

In [ ]:
_ = plt.scatter(X, y)

In [ ]:
_ = plt.scatter(X_test, y_test)

In [ ]:
# JAX logical execution plan
print(jax.make_jaxpr(update)(jnp.ones(X.shape[1]), X, y, 0.01))

In [ ]:
# X.shape -> (n_samples, n_features)

lr_model = LinearRegression(X.shape[1])
lr = 0.01
epochs = 101

print(f"Initial Test MSE: {lr_model.mse(X_test,y_test)}")

# silent: logging verbosity
lr_model.fit(X,y, epochs, lr, silent=False)

print(f"Final Test MSE: {lr_model.mse(X_test,y_test)}")

print(f"Final parameters: {lr_model.weights}")